In [1]:
import csv
import os
import gc
import numpy as np
import tensorflow as tf
from random import randint
from tensorflow import keras
from keras.utils import np_utils, plot_model, to_categorical
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, concatenate

In [14]:
# returns train, inference_encoder and inference_decoder models
def define_models(dim_p, n_units):
	# define training encoder
	enc_in_o = Input(shape=(None, 1))
	enc_in_q = Input(shape=(None, 1))
	enc_in_p = Input(shape=(None, dim_p))
	encoder_inputs = concatenate([enc_in_o, enc_in_q, enc_in_p])
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	
	# define training decoder
	dec_in_o = Input(shape=(None, 1))
	dec_in_q = Input(shape=(None, 1))
	dec_in_p = Input(shape=(None, dim_p))
	decoder_inputs = concatenate([dec_in_o, dec_in_q, dec_in_p])
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	dec_dense_o = Dense(1, activation='relu', name='tr_out_o')
	dec_dense_q = Dense(1, activation='sigmoid', name='tr_out_q')
	dec_dense_p = Dense(dim_p, activation='softmax', name='tr_out_p')
	#out_o = Dense(1, activation='relu', name='tr_out_o')(decoder_outputs)#act relu
	#out_q = Dense(1, activation='sigmoid', name='tr_out_q')(decoder_outputs)
	#out_p = Dense(dim_p, activation='softmax', name='tr_out_p')(decoder_outputs)
	out_o = dec_dense_o(decoder_outputs)
	out_q = dec_dense_q(decoder_outputs)
	out_p = dec_dense_p(decoder_outputs)
	
	model = Model([enc_in_o, enc_in_q, enc_in_p, dec_in_o, dec_in_q, dec_in_p], [out_o, out_q, out_p])
	
	# define inference encoder
	encoder_model = Model([enc_in_o, enc_in_q, enc_in_p], encoder_states)
	
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	#out_o = TimeDistributed(Dense(1, activation='relu'))(decoder_outputs)#act relu
	#out_q = TimeDistributed(Dense(1, activation='sigmoid'))(decoder_outputs)
	#out_p = TimeDistributed(Dense(dim_p, activation='softmax'))(decoder_outputs)
	out_o = dec_dense_o(decoder_outputs)
	out_q = dec_dense_q(decoder_outputs)
	out_p = dec_dense_p(decoder_outputs)

	decoder_model = Model([dec_in_o, dec_in_q, dec_in_p] + decoder_states_inputs, [out_o, out_q, out_p] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model


# generate target given source sequence
def predict_sequence(infenc, infdec, src_o, src_q, src_p, n_steps, cardinality):
	# encode
	state = infenc.predict([src_o, src_q, src_p])
	# start of sequence input
	target_o = np.array([-1]).reshape(1, 1, 1)
	target_q = np.array([-1]).reshape(1, 1, 1)
	#target_p = 0
	target_p = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		#print(target_o.shape)
		#print(target_q.shape)
		#print(target_p.shape)
		#print(state[0].shape)
		o, q, p, h, c = infdec.predict([target_o, target_q, target_p] + state)
		#print(a)
		# store prediction
		output.append(o[0,0,:])
		output.append(q[0,0,:])
		output.append(p[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_o = o
		target_q = q
		target_p = p
	return np.array(output)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [np.argmax(vector) for vector in encoded_seq]

In [3]:
#load data
stream_list = []

for directory, subdirectories, files in os.walk('data'):
    for file in files:
        with open('data/' + file, 'r') as f: 
            reader = csv.reader(f)
            sub_list = [list(map(float,rec)) for rec in csv.reader(f, delimiter=',')]
            stream_list = stream_list + sub_list

In [4]:
#create seperate data structures for each variable (offset, quarterlength, pitch)
#normalise offset and quarterlength
offs = []
qlngth = []
ptch = []

offsb = max(element[0] for element in stream_list)
qlngthb = max(element[1] for element in stream_list)
#ptchb = 127.0

for row in stream_list:
    offs.append(row[0]/offsb)
    qlngth.append(row[1]/qlngthb)
    ptch.append(row[2])

In [ ]:
del stream_list
gc.collect()

20

In [5]:
#divide the sets in sequences of specific length 
dtlngth=len(offs)
seq_length = 4#100 groups of 3

dataX1_o = []
dataX1_q = []
dataX1_p = []
dataX2_o = []
dataX2_q = []
dataX2_p = []

for i in range(0, dtlngth - seq_length + 1, 1):
	seq_in_o = offs[i:i + seq_length]
	seq_in_q = qlngth[i:i + seq_length]
	seq_in_p = ptch[i:i + seq_length]

	# create padded input target sequence
	target_in_o = [-1] + seq_in_o[:-1]
	target_in_q = [-1] + seq_in_q[:-1]
	target_in_p = [0] + seq_in_p[:-1]
	
	src_encoded = to_categorical(seq_in_p, num_classes=127+1)
	tar2_encoded = to_categorical(target_in_p, num_classes=127+1)
	
	dataX1_o.append([offs for offs in seq_in_o])
	dataX1_q.append([qlngth for qlngth in seq_in_q])
	dataX1_p.append(src_encoded)
	dataX2_o.append([offs for offs in target_in_o])
	dataX2_q.append([qlngth for qlngth in target_in_q])
	dataX2_p.append(tar2_encoded)

n_patterns = len(dataX1_p)
print ("Total Patterns: ", n_patterns)

Total Patterns:  32385


In [6]:
#reshape inputs to be [samples, time steps, features], one hot encode pitch output
dataX1_o = np.reshape(dataX1_o, (n_patterns, seq_length, 1))
dataX1_q = np.reshape(dataX1_q, (n_patterns, seq_length, 1))

dataX2_o = np.reshape(dataX2_o, (n_patterns, seq_length, 1))
dataX2_q = np.reshape(dataX2_q, (n_patterns, seq_length, 1))

In [ ]:
del offs
del qlngth
del ptch
gc.collect()

In [15]:
# configure problem
n_features = np.array(dataX2_p).shape[2]
n_steps_out = seq_length
# define model
train, infenc, infdec = define_models(n_features, 256)
train.compile(optimizer='adam', loss={'tr_out_o': 'mse', 'tr_out_q': 'mse', 'tr_out_p': 'categorical_crossentropy'},
 metrics={'tr_out_o': 'mean_squared_error', 'tr_out_q': 'mean_squared_error', 'tr_out_p': 'accuracy'})

In [8]:
# spot check some examples
for _ in range(1):
    i = randint(1, 1000)
    X1_o = np.reshape(dataX1_o[i], (1, seq_length, 1))
    X1_q = np.reshape(dataX1_q[i], (1, seq_length, 1))
    X1_p = np.reshape(dataX1_p[i], (1, seq_length, n_features))
    target = predict_sequence(infenc, infdec, X1_o, X1_q, X1_p, n_steps_out, n_features)
    for j in range(seq_length):
        print('X_o=%s, y_o=%s, X_q=%s, y_q=%s, X_p=%s, y_p=%s' % (dataX1_o[i][j]*offsb, target[3*j]*offsb, dataX1_q[i][j]*qlngthb, target[3*j+1]*qlngthb, one_hot_decode([dataX1_p[i][j]]), one_hot_decode([target[3*j+2]])))
    print()

X_o=[76.75], y_o=[0.], X_q=[0.25], y_q=[19.003607], X_p=[72], y_p=[69]
X_o=[77.], y_o=[0.], X_q=[0.5], y_q=[18.842424], X_p=[69], y_p=[32]
X_o=[77.5], y_o=[0.], X_q=[0.25], y_q=[18.741777], X_p=[67], y_p=[101]
X_o=[77.5], y_o=[0.], X_q=[0.33333333], y_q=[18.682713], X_p=[38], y_p=[117]

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
#divide data in train and validation sets
dataX1_o_tr = dataX1_o[:-1000]
dataX1_o_v = dataX1_o[-1000:]
dataX2_o_tr = dataX2_o[:-1000]
dataX2_o_v = dataX2_o[-1000:]

dataX1_q_tr = dataX1_q[:-1000]
dataX1_q_v = dataX1_q[-1000:]
dataX2_q_tr = dataX2_q[:-1000]
dataX2_q_v = dataX2_q[-1000:]

dataX1_p_tr = dataX1_p[:-1000]
dataX1_p_v = dataX1_p[-1000:]
dataX2_p_tr = dataX2_p[:-1000]
dataX2_p_v = dataX2_p[-1000:]

In [16]:
# train model
train.fit([dataX1_o_tr, dataX1_q_tr, np.array(dataX1_p_tr), dataX2_o_tr, dataX2_q_tr, np.array(dataX2_p_tr)], [dataX1_o_tr, dataX1_q_tr, np.array(dataX1_p_tr)], epochs=10)

Epoch 1/10
981/981 [==============================] - 103s 105ms/step - loss: 0.8550 - tr_out_o_loss: 0.0091 - tr_out_q_loss: 0.0035 - tr_out_p_loss: 0.8424 - tr_out_o_mean_squared_error: 0.0091 - tr_out_q_mean_squared_error: 0.0035 - tr_out_p_accuracy: 0.7594
Epoch 2/10
981/981 [==============================] - 102s 104ms/step - loss: 0.0448 - tr_out_o_loss: 0.0026 - tr_out_q_loss: 0.0013 - tr_out_p_loss: 0.0409 - tr_out_o_mean_squared_error: 0.0026 - tr_out_q_mean_squared_error: 0.0013 - tr_out_p_accuracy: 0.9923
Epoch 3/10
981/981 [==============================] - 141s 143ms/step - loss: 0.0205 - tr_out_o_loss: 0.0015 - tr_out_q_loss: 0.0011 - tr_out_p_loss: 0.0179 - tr_out_o_mean_squared_error: 0.0015 - tr_out_q_mean_squared_error: 0.0011 - tr_out_p_accuracy: 0.9969
Epoch 4/10
981/981 [==============================] - 166s 169ms/step - loss: 0.0152 - tr_out_o_loss: 0.0011 - tr_out_q_loss: 8.7780e-04 - tr_out_p_loss: 0.0132 - tr_out_o_mean_squared_error: 0.0011 - tr_out_q_mean_sq

In [ ]:
def generatorex(features1, features2, features3, labels1, labels2, labels3, batch_size):
    # Create empty arrays to contain batch of features and labels# 
    batch_features1 = np.zeros((batch_size, 4, 1))
    batch_features2 = np.zeros((batch_size, 4, 1))
    batch_features3 = np.zeros((batch_size, 4, 128))
    batch_labels1 = np.zeros((batch_size, 4, 1))
    batch_labels2 = np.zeros((batch_size, 4, 1))
    batch_labels3 = np.zeros((batch_size, 4, 128))
    i = 0
    while True:
        for b in range(batch_size):
            batch_features1[b] = features1[i]
            batch_features2[b] = features2[i]
            batch_features3[b] = features3[i]
            batch_labels1[b] = labels1[i]
            batch_labels2[b] = labels2[i]
            batch_labels3[b] = labels3[i]
            i += 1
            if (i == len(features1)):
                i=0
        #print(batch_features, batch_labels)
        yield [batch_features1, batch_features2, batch_features3, batch_labels1, batch_labels2, batch_labels3], [batch_features1, batch_features2, batch_features3]

In [ ]:
train.fit(generatorex(dataX1_o, dataX1_q, np.array(dataX1_p), dataX2_o, dataX2_q, np.array(dataX2_p), 180), epochs = 10, steps_per_epoch=540)


In [ ]:
infdec.summary()

In [17]:
# spot check some examples
for _ in range(10):
    i = randint(1, 1000)
    X1_o = np.reshape(dataX1_o_v[i], (1, seq_length, 1))
    X1_q = np.reshape(dataX1_q_v[i], (1, seq_length, 1))
    X1_p = np.reshape(dataX1_p_v[i], (1, seq_length, n_features))
    target = predict_sequence(infenc, infdec, X1_o, X1_q, X1_p, n_steps_out, n_features)
    for j in range(seq_length):
        print('X_o=%s, y_o=%s, X_q=%s, y_q=%s, X_p=%s, y_p=%s' % (dataX1_o_v[i][j]*offsb, target[3*j]*offsb, dataX1_q_v[i][j]*qlngthb, target[3*j+1]*qlngthb, 
        one_hot_decode([dataX1_p_v[i][j]]), one_hot_decode([target[3*j+2]])))
    print()

X_o=[356.], y_o=[366.22577], X_q=[3.5], y_q=[1.7726686], X_p=[76], y_p=[76]
X_o=[357.5], y_o=[355.2946], X_q=[1.5], y_q=[1.5769769], X_p=[69], y_p=[69]
X_o=[357.5], y_o=[337.3426], X_q=[1.5], y_q=[1.2794383], X_p=[73], y_p=[73]
X_o=[357.5], y_o=[329.82224], X_q=[1.5], y_q=[0.5931359], X_p=[76], y_p=[76]

X_o=[429.5], y_o=[413.27713], X_q=[2.], y_q=[2.53242], X_p=[78], y_p=[78]
X_o=[429.5], y_o=[410.4667], X_q=[0.5], y_q=[0.90786743], X_p=[62], y_p=[62]
X_o=[429.66666667], y_o=[430.47012], X_q=[2.], y_q=[1.1060033], X_p=[69], y_p=[69]
X_o=[429.66666667], y_o=[412.40094], X_q=[2.], y_q=[0.7233661], X_p=[72], y_p=[72]

X_o=[365.5], y_o=[335.17642], X_q=[1.], y_q=[1.2596895], X_p=[61], y_p=[61]
X_o=[366.], y_o=[338.3965], X_q=[0.75], y_q=[0.67638385], X_p=[28], y_p=[28]
X_o=[366.5], y_o=[353.43735], X_q=[1.33333333], y_q=[1.0144595], X_p=[62], y_p=[62]
X_o=[367.], y_o=[362.9392], X_q=[0.75], y_q=[0.962466], X_p=[30], y_p=[30]

X_o=[509.5], y_o=[493.72354], X_q=[2.], y_q=[3.7351773], X_p=[6

In [ ]:
print(one_hot_decode([dataX1_p[i][j]]))


[43]


In [ ]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def strided_axis0(a, fillval, L): # a is 1D array
    a_ext = np.concatenate(( np.full(L-1,fillval) ,a))
    n = a_ext.strides[0]
    strided = np.lib.stride_tricks.as_strided     
    return strided(a_ext, shape=(a.shape[0],L), strides=(n,n))

#if (rolling_window(np.asarray(offs), 6).all == np.asarray(dataX1_o).all):
#print(np.asarray(dataX1_o).shape)
#print(rolling_window(np.asarray(offs), 6).shape)
#print(dataX1_o[5460][2]*offsb)

ptch_fs = to_categorical(ptch, num_classes=127+1)
ptch_fs = rolling_window(np.asarray(ptch_fs), 6)
#ptch_fs = to_categorical(ptch_fs, num_classes=127+1)

#print (ptch_fs.shape)
#print (np.asarray(dataX1_p).shape)

#one_hot_decode(ptch_fs) == one_hot_decode(dataX1_p)
ptch_fs[-1]

#strided_axis0(np.asarray(ptch), 0, 6)